In [10]:
import numpy as np

def rodrigues_rotation(s, phi):
    s = s / np.linalg.norm(s)
    I = np.eye(3)
    s_hat = np.array([
        [0, -s[2], s[1]],
        [s[2], 0, -s[0]],
        [-s[1], s[0], 0]
    ])
    
    s_hat_sq = np.dot(s_hat, s_hat)
    R = I + np.sin(phi) * s_hat + (1 - np.cos(phi)) * s_hat_sq
    
    return R

# z-axis is the rotation axis
axis = np.array([0, 0, 1])
phi = np.pi / 2
R = rodrigues_rotation(axis, phi)

# Verify the formula
print(f"Verify the formula (True/False): {np.isclose(np.cos(phi), (np.trace(R)- 1)/2)}\n")

points_before = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1], # should stay the same
    [1, 1, 1]
])
points_after = (R @ points_before.T).T

# Show points before and after
print("Points Before Rotation:")
print(points_before)
print("\nPoints After Rotation:")
print(np.round(points_after, 4))

Verify the formula (True/False): True

Points Before Rotation:
[[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 1]]

Points After Rotation:
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]
 [-1.  1.  1.]]


In [15]:
def extract_axis_angle(R):
    # Calculate phi using trace
    phi = np.arccos(0.5 * (np.trace(R) - 1))
    
    # find s using R - R_T = 2*sin(phi)*s_hat
    s_hat_2sin = R - R.T
    s_hat = s_hat_2sin / (2*np.sin(phi))
    s1 = s_hat[2, 1]
    s2 = s_hat[0, 2]
    s3 = s_hat[1, 0]

    s = np.array([s1, s2, s3])    
    return s, phi
calculated_s, calculated_phi = extract_axis_angle(R)
print(f"Calculated s: {calculated_s}, Calculated phi: {calculated_phi}")
print(f"Verify correctness (True/False): {np.allclose(calculated_s, axis) and np.isclose(calculated_phi, phi)}\n")

Calculated s: [0. 0. 1.], Calculated phi: 1.5707963267948966
Verify correctness (True/False): True



In [ ]:
def find_best_transform(u, v):
    centroid_u = np.mean(u, axis=0)
    centroid_v = np.mean(v, axis=0)
    u_centered = u - centroid_u
    v_centered = v - centroid_v

    # Covariance Matrix H
    H = np.dot(u_centered.T, v_centered)

    # Use SVD here to find optimal rotation
    U, _, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # Find translation
    T = centroid_v - np.dot(R, centroid_u)

    # Assemble E
    E = np.eye(3)
    E[:2, :2] = R
    E[:2, 2] = T
    
    return E

u_pts = np.array([[-3, 0],[1, 1],[1, 0],[1, -1]])
v_pts = np.array([[0, 3],[1, 0],[0, 0],[-1, 0]])

best_E = find_best_transform(u_pts, v_pts)
print(f"Optimal E:\n{best_E}")

Optimal E:
[[ 0.    1.    0.  ]
 [-1.    0.    0.75]
 [ 0.    0.    1.  ]]
